In [9]:
import numpy as np
import pandas as pd
import ddm
import os 
import helper
import models

In [15]:
for model in [1, 2, 3]:
    for folder in ["full_data"]:#, "cross_validation_8"]:
        helper.merge_csv("../model_fit_results/model_%i/best_fit_parameters/%s" % (model, folder))

../model_fit_results/model_1/best_fit_parameters/full_data\subj_all.csv
../model_fit_results/model_2/best_fit_parameters/full_data\subj_all.csv
../model_fit_results/model_3/best_fit_parameters/full_data\subj_all.csv


In [11]:
def get_model_measures(model, condition):
    sol = model.solve(condition)
    return condition["tta_condition"], condition["d_condition"], sol.prob_correct(), sol.mean_decision_time()

def get_model_rt_distr(model, condition, kind="cdf"):
    sol = model.solve(condition)
    return pd.DataFrame({"tta_condition": condition["tta_condition"],
                         "d_condition":  condition["d_condition"],
                         "t": model.t_domain(),
                         "rt_corr_distr": (sol.cdf_corr() if kind=="cdf" else sol.pdf_corr())/sol.prob_correct()})

def initialize_model(model_no, param_set):
    # all three models use Gaussian non-decision time
    overlay = models.OverlayNonDecisionGaussian(ndt_location=param_set.ndt_location, ndt_scale=param_set.ndt_scale)
    drift = models.DriftTtaDistanceDynamic(alpha=param_set.alpha, beta=param_set.beta, theta=param_set.theta)

    if model_no == 1:
        bound = models.BoundCollapsingTta(b_0=param_set.b_0, k=param_set.k, tta_crit=param_set.tta_crit)
    elif model_no == 2:
        bound = models.BoundCollapsingTtaDistancev2(b_0=param_set.b_0, k=param_set.k,
                                                    beta_boundary=param_set.beta_boundary,
                                                    theta_boundary=param_set.theta_boundary)
    elif model_no == 3:
        bound = models.BoundCollapsingGeneralizedGap(b_0=param_set.b_0, k=param_set.k,
                                                     beta=param_set.beta, theta=param_set.theta)

    model = ddm.Model(name="Model %i" % model_no, drift=drift, bound=bound, overlay=overlay,
                      noise=ddm.NoiseConstant(noise=1), T_dur=models.ModelStaticDriftFixedBounds.T_dur)

    return model

def simulate_model(model_no, param_set, conditions, ret="measures"):
    """
    Set ret to "measures" or "rt_cdf" or "rt_pdf" for saving p_turn and mean RT or RT CDF or RT PDF
    """
    model = initialize_model(model_no, param_set)

    if ret=="measures":
        sim_result = pd.DataFrame([get_model_measures(model, condition) for condition in conditions],
                                  columns=["tta_condition", "d_condition", "is_go_decision", "RT"])
    else:
        sim_result = pd.concat([get_model_rt_distr(model, condition, kind=ret[-3:]) for condition in conditions])
    sim_result["subj_id"] = param_set.subj_id
    return sim_result

In [12]:
def save_sim_results(model_no, file_name, conditions=None, cross_validation=False, ret="measures"):
    path = "../model_fit_results/model_%i/best_fit_parameters/" % model_no
    file_path = os.path.join(path, file_name)
    parameters = pd.read_csv(file_path).rename(columns={"tta": "tta_condition", "d": "d_condition"})

    # print(parameters)

    sim_results = [simulate_model(model_no, param_set,
                                  [param_set[["tta_condition", "d_condition"]].to_dict()] if cross_validation else conditions,
                                  ret=ret)
                   for idx, param_set in parameters.iterrows()]

    sim_results = pd.concat(sim_results)

    sim_results.to_csv(os.path.join("../model_fit_results/model_%i/simulation_results/" % model_no,
                                    file_name.replace("parameters_fitted", ret)),
                       index=False)

# Simulate model trained on full data

In [13]:
conditions = [{"tta_condition": tta, "d_condition": d}
                       for tta in np.linspace(4.5,7.5,13)
                       for d in np.linspace(50,110,13)]

In [12]:
save_sim_results(1, "full_data_parameters_fitted.csv", conditions, cross_validation=False, ret="measures")

In [16]:
for model_no in [1, 2, 3]:
    save_sim_results(model_no, "full_data_parameters_fitted.csv", conditions, cross_validation=False, ret="measures")

In [ ]:
for model_no in [1, 2, 3]:
    save_sim_results(model_no, "full_data_parameters_fitted.csv", conditions, cross_validation=False, ret="rt_cdf")

# Simulating cross-validated model

In [13]:
conditions = [{"tta_condition": tta, "d_condition": d}
                       for tta in [4,5,6]
                       for d in [90,120,150]]

In [14]:
for model_no in [1, 2, 3]:
    save_sim_results(model_no, "cross_validation_8_parameters_fitted.csv", cross_validation=True, ret="measures", conditions=conditions)